<a href="https://colab.research.google.com/github/ghirailghiro/GPU_Computing_Project/blob/10-add-sequential-algorithm/Gradient_First_Step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Cuda plugin

!pip install nvcc4jupyter
%load_ext nvcc4jupyter
!nvidia-smi

# plugin for cpp sintax highlighting

!wget -O cpp_plugin.py https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
%load_ext cpp_plugin

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmph1nuyeur".
Tue Jul 30 20:48:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                 

In [2]:
!sudo apt update
!sudo apt install -y build-essential
!sudo apt install -y libopencv-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [861 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Ge

In [3]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!ls '/content/drive/My Drive/GPU Computing/human detection dataset'

0  1


In [ ]:
!git clone --recursive https://github.com/YoungYoung619/pedestrian-detection-in-hazy-weather.git

In [ ]:
%%writefile my_cpp_code.cpp
#include <opencv2/opencv.hpp>
#include <fstream>
#include <sstream>
#include <iostream>
#include <vector>
#include <filesystem>
#include <cmath>

namespace fs = std::filesystem;

void saveDescriptorAsCSVHeader(const std::vector<int>& descriptor, const std::string& filename, const std::string& label) {
    std::ofstream file(filename, std::ios::app);
    if (!file.is_open()) {
        std::cerr << "Error: Unable to open file " << filename << " for writing." << std::endl;
        return;
    }

    // Write the descriptor to the CSV file
    for (size_t i = 0; i < descriptor.size(); ++i) {
        file << "x" << descriptor[i];
        if (i < descriptor.size() - 1) {
            file << ",";
        }
    }
    file << "," << label << "\n";
    file.close();
}


void saveDescriptorAsCSV(const float* descriptor, int descriptorSize, const std::string& filename, int label) {
    std::ofstream file(filename, std::ios::app);
    if (!file.is_open()) {
        std::cerr << "Error: Unable to open file " << filename << " for writing." << std::endl;
        return;
    }

    // Write the descriptor to the CSV file
    for (int i = 0; i < descriptorSize; ++i) {
        file << descriptor[i];
        if (i < descriptorSize - 1) {
            file << ",";
        }
    }
    file << "," << label << "\n";
    file.close();
}

void computeGradients(const cv::Mat& image, std::vector<float>& magnitude, std::vector<float>& orientation, std::vector<float>& histograms, int cellSize) {
    magnitude.clear();
    orientation.clear();
    histograms.clear();
    std::cout << "Entering computeGradients"<< std::endl;


    // Assuming image dimensions are reasonable for a grid of threads
    int width = image.cols;
    int height = image.rows;

    // Compute gradients, magnitude, and orientation

    for (int idy = 0; idy < height; ++idy) {
        for (int idx = 0; idx < width; ++idx) {

            float G_x = 0, G_y = 0;
            if (idx > 0 && idx < width - 1) {
                G_x = static_cast<float>(image.at<uchar>(idy, idx + 1)) - static_cast<float>(image.at<uchar>(idy, idx - 1));
            }
            if (idy > 0 && idy < height - 1) {
                G_y = static_cast<float>(image.at<uchar>(idy + 1, idx)) - static_cast<float>(image.at<uchar>(idy - 1, idx));
            }

            float mag = std::sqrt(G_x * G_x + G_y * G_y);
            float orient = std::atan2(G_y, G_x);


            magnitude.push_back(mag);
            orientation.push_back(orient);


            // Compute histogram bin for the current gradient
            int cellX = idx / cellSize;
            int cellY = idy / cellSize;
            int histIndex = cellY * (width / cellSize) + cellX;
            int numBins = 9; // Assuming 9 orientation bins
            float binWidth = M_PI / numBins;
            int bin = std::floor((orient + M_PI) / binWidth);
            if (bin == numBins) bin = 0; // Wrap around
            histograms[histIndex * numBins + bin] += mag;
        }

    }

    // Normalization
    for (size_t i = 0; i < histograms.size(); i += 9) {
        float sum = 0.0f;
        for (int j = 0; j < 9; ++j) {
            sum += histograms[i + j] * histograms[i + j];
        }
        sum = std::sqrt(sum);
        for (int j = 0; j < 9; ++j) {
            histograms[i + j] /= (sum + 1e-6); // Small constant added to avoid division by zero
        }
    }

}

float* computeDescriptors(const std::string& image_path, int& descriptorSize) {


    cv::Mat imageBeforeResize = cv::imread(image_path, cv::IMREAD_GRAYSCALE);
    cv::Mat image;
    cv::resize(imageBeforeResize, image, cv::Size(224, 224)); // Resize to standard size
    int cellSize = 64;
    int numCellsX = image.cols / cellSize;
    int numCellsY = image.rows / cellSize;
    int descriptorSizeDimension = (numCellsY - 1) * (numCellsX - 1) * 2 * 2 * 9; // Size of descriptor array
    descriptorSize = (numCellsY - 1) * (numCellsX - 1) * 2 * 2 * 9; // Size of descriptor array

    float* descriptorArray = new float[descriptorSizeDimension];
    if (image.empty()) {
        std::cerr << "Failed to load image." << std::endl;
        return descriptorArray;
    }



    std::vector<float> magnitude, orientation;

    size_t histSize = numCellsX * numCellsY * 9;

    // Declare a vector to store the histogram data
    std::vector<float> histograms(histSize);
    computeGradients(image, magnitude, orientation, histograms, cellSize);
    // Block Formation and Descriptor Computation
    image.release();
    magnitude.clear();
    orientation.clear();

    int index = 0; // Index to keep track of position in the descriptor array

    for (int i = 0; i < numCellsY - 1; ++i) {
        for (int j = 0; j < numCellsX - 1; ++j) {
            // Concatenate histograms of four cells into a block
            for (int y = i; y < i + 2; ++y) {
                for (int x = j; x < j + 2; ++x) {
                    for (int k = 0; k < 9; ++k) {
                        // Store histogram value directly into the descriptor array
                        //std::cout << "k : "<< k << " calc: "  << histograms[(y * numCellsX + x) * 9 + k] << std::endl;
                        descriptorArray[index++] = histograms[(y * numCellsX + x) * 9 + k];
                    }
                }
            }
        }
    }
    histograms.clear();
    return descriptorArray;
}

int main() {
    std::string folder_path = "/content/drive/My Drive/GPU Computing/human detection dataset/1"; // Change this to your folder path
    std::vector<int> header;
    for (int i = 1; i <= 144; ++i) {
        header.push_back(i);
    }
    saveDescriptorAsCSVHeader(header, "descriptor.csv", "label");
    header.clear();

    // People present class
    for (const auto& entry : fs::directory_iterator(folder_path)) {
        std::string file_path = entry.path().string();
        std::cout << "Processing image: " << file_path << std::endl;

        int label = 1; // Assuming the label is 1 for the class "People present"
        int descriptorSize = 0;
        float* descriptor = computeDescriptors(file_path, descriptorSize);
        std::cout << "Dimension descriptor: " << descriptorSize << std::endl;

        saveDescriptorAsCSV(descriptor, descriptorSize, "descriptor.csv", label);
        delete[] descriptor; // Don't forget to deallocate memory
    }
    return 0;
}


Overwriting my_cpp_code.cpp


In [ ]:
!g++ my_cpp_code.cpp -o my_cpp_program `pkg-config --cflags --libs opencv4`


In [ ]:
!./my_cpp_program


Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/76.png
Entering computeGradients
Dimension descriptor: 144
malloc(): unaligned tcache chunk detected


In [12]:
%%writefile gradient_computation.cu
#include <opencv2/opencv.hpp>
#include <fstream>
#include <sstream>
#include <iostream>
#include <vector>
#include <filesystem>
#include <chrono>

namespace fs = std::filesystem;

void saveDescriptorAsCSVHeader(const std::vector<int>& descriptor, const std::string& filename, const std::string& label) {
    std::ofstream file(filename, std::ios::app);
    if (!file.is_open()) {
        std::cerr << "Error: Unable to open file " << filename << " for writing." << std::endl;
        return;
    }

    // Write the descriptor to the CSV file
    for (size_t i = 0; i < descriptor.size(); ++i) {
        file << "x" <<descriptor[i];
        if (i < descriptor.size() - 1) {
            file << ",";
        }
    }
    file << "," << label <<","<< "Exec Time" << "\n";
    file.close();
}


void saveDescriptorAsCSV(const std::vector<float>& descriptor, const std::string& filename, int label,  double executionTime) {
    std::ofstream file(filename, std::ios::app);
    if (!file.is_open()) {
        std::cerr << "Error: Unable to open file " << filename << " for writing." << std::endl;
        return;
    }

    // Write the descriptor to the CSV file
    for (size_t i = 0; i < descriptor.size(); ++i) {
        file << descriptor[i];
        if (i < descriptor.size() - 1) {
            file << ",";
        }
    }
    file << "," << label << "," << executionTime << "\n";
    file.close();
}

__global__ void computeGradients(unsigned char* image, float *d_magnitude, float *d_orientation, float *d_histograms, int width, int height, int cellSize) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int idy = blockIdx.y * blockDim.y + threadIdx.y;
    int indexCurrent = idy * width + idx;

    if (idx >= width || idy >= height) return; // Boundary check

    float G_x = 0;
    if (idx > 0 && idx < width - 1) {
        G_x = (float)image[idy * width + (idx + 1)] - (float)image[idy * width + (idx - 1)];
    }

    float G_y = 0;
    if (idy > 0 && idy < height - 1) {
        G_y = (float)image[(idy + 1) * width + idx] - (float)image[(idy - 1) * width + idx];
    }

    d_magnitude[indexCurrent] = sqrtf(G_x * G_x + G_y * G_y);
    d_orientation[indexCurrent] = atan2f(G_y, G_x);

    // Compute histogram bin for the current gradient
    int cellX = idx / cellSize;
    int cellY = idy / cellSize;
    int histIndex = cellY * (width / cellSize) + cellX;
    int numBins = 9; // Assuming 9 orientation bins
    float binWidth = M_PI / numBins;
    int bin = floor((d_orientation[indexCurrent] + M_PI) / binWidth);
    if (bin == numBins) bin = 0; // Wrap around
    atomicAdd(&d_histograms[histIndex * numBins + bin], d_magnitude[indexCurrent]);

    /*if((idx == 2 && idy == 2) || (idx==300 & idy == 300)){
      printf("Value for idx %d e idy %d - bin value: %d - orientation value %f \n", idx, idy, bin,d_orientation[indexCurrent] + M_PI);
    }*/
}

std::vector<float> computeDescriptors(const std::string& image_path, double& executionTime){
        // Example: Load an image using OpenCV
    cv::Mat imageBeforeResize = cv::imread(image_path, cv::IMREAD_GRAYSCALE);
    cv::Mat image;
    cv::resize(imageBeforeResize, image, cv::Size(224, 224)); // Resize to standard size
    if(image.empty()) {
        std::cerr << "Failed to load image." << std::endl;
        return std::vector<float>();
    }

    unsigned char* d_image;
    size_t imageSize = image.total() * image.elemSize();
    cudaMalloc(&d_image, imageSize);
    cudaMemcpy(d_image, image.data, imageSize, cudaMemcpyHostToDevice);
    size_t sizeInBytes = image.total() * sizeof(float);
    float* d_magnitude;
    cudaError_t status = cudaMalloc((void **)&d_magnitude, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error (e.g., printing an error message and exiting)
        fprintf(stderr, "cudaMalloc failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    status = cudaMemset(d_magnitude, 0, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMemset failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    sizeInBytes = image.total() * sizeof(float);
    float* d_orientation;
    status = cudaMalloc((void **)&d_orientation, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error (e.g., printing an error message and exiting)
        fprintf(stderr, "cudaMalloc failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    status = cudaMemset(d_orientation, 0, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMemset failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }

    // Assuming image dimensions are reasonable for a grid of threads
    dim3 blockSize(16, 16);
    dim3 gridSize((image.cols + blockSize.x - 1) / blockSize.x,
                  (image.rows + blockSize.y - 1) / blockSize.y);

    // Allocate memory for histograms
    int cellSize = 64;
    int numCellsX = image.cols / cellSize;
    int numCellsY = image.rows / cellSize;
    size_t histSize = numCellsX * numCellsY * 9 * sizeof(float);
    float* d_histograms;
    status = cudaMalloc((void **)&d_histograms, histSize);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMalloc failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    status = cudaMemset(d_histograms, 0, histSize);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMemset failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    auto start = std::chrono::high_resolution_clock::now();
    // Launch the kernel
    computeGradients<<<gridSize, blockSize>>>(d_image, d_magnitude, d_orientation, d_histograms, image.cols, image.rows, cellSize);
    cudaDeviceSynchronize();
    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed = end - start;
    executionTime = elapsed.count();

    // Transfer histogram data from device to host
    float* h_histograms = new float[numCellsX * numCellsY * 9];
    cudaMemcpy(h_histograms, d_histograms, histSize, cudaMemcpyDeviceToHost);

    /*for (int i = 0; i < numCellsX * numCellsY; ++i) {
        for (int j = 0; j < 9; ++j) {
            std::cout << "Cell " << i << ", Bin " << j << ": " << h_histograms[i * 9 + j] << std::endl;
        }
    }*/
    // Cleanup

    // Normalization
    for (int i = 0; i < numCellsX * numCellsY; ++i) {
        float sum = 0.0f;
        for (int j = 0; j < 9; ++j) {
            sum += h_histograms[i * 9 + j] * h_histograms[i * 9 + j];
        }
        sum = sqrtf(sum);
        for (int j = 0; j < 9; ++j) {
            h_histograms[i * 9 + j] /= (sum + 1e-6); // Small constant added to avoid division by zero
        }
    }

    // Block Formation and Descriptor Computation
    std::vector<float> descriptor;
    for (int i = 0; i < numCellsY - 1; ++i) {
        for (int j = 0; j < numCellsX - 1; ++j) {
            // Concatenate histograms of four cells into a block
            for (int y = i; y < i + 2; ++y) {
                for (int x = j; x < j + 2; ++x) {
                    for (int k = 0; k < 9; ++k) {
                        descriptor.push_back(h_histograms[(y * numCellsX + x) * 9 + k]);
                    }
                }
            }
        }
    }
    cudaFree(d_image);
    cudaFree(d_magnitude);
    cudaFree(d_orientation);
    delete[] h_histograms;
    cudaFree(d_histograms);

    return descriptor;
}

int main(int argc, char** argv) {
     if (argc != 6) {
        std::cerr << "Usage: " << argv[0] << " <cellSize> <blockSize> <numBins> <outputFile> <dimofimage>" << std::endl;
        return 1;
    }

    int cellSize = std::stoi(argv[1]);
    int blockSize = std::stoi(argv[2]);
    int numBins = std::stoi(argv[3]);
    std::string outputFile = argv[4];
    int dimofimage = std::stoi(argv[5]);//224
    int numCellsX = dimofimage / cellSize;
    int numCellsY = dimofimage / cellSize;
    int descriptorSizeDimension = (numCellsY - blockSize + 1) * (numCellsX - blockSize + 1) * blockSize * blockSize * numBins;

    std::string folder_path = "/content/drive/My Drive/GPU Computing/human detection dataset/1"; // Change this to your folder path
    std::vector<int> header;
    for (int i=1; i <= descriptorSizeDimension; ++i){
      header.push_back(i);
    }
    saveDescriptorAsCSVHeader(header, "descriptor.csv", "label");
    header.clear();
    //People present class
    for (const auto& entry : fs::directory_iterator(folder_path)) {
        std::string file_path = entry.path().string();
        std::cout << "Processing image: " << file_path << std::endl;

        double executionTime = 0.0;
        std::vector<float> descriptor = computeDescriptors(file_path, executionTime);
        if (descriptor.empty()) {
            std::cout << "Vector is empty" << std::endl;
        } else {
            int label = 1;
            std::vector<float> descriptor1;
            int length = static_cast<int>(descriptor.size());
            std::cout << "Dimension of descriptor: " << length << std::endl;
            descriptor1.push_back(length);
            saveDescriptorAsCSV(descriptor, "descriptor.csv", label, executionTime);
            descriptor.clear(); // Clear the vector
            descriptor1.clear();
        }
    }

      //Not people present class
      folder_path = "/content/drive/My Drive/GPU Computing/human detection dataset/0";
      for (const auto& entry : fs::directory_iterator(folder_path)) {
        std::string file_path = entry.path().string();
        std::cout << "Processing image: " << file_path << std::endl;

        double executionTime = 0.0;
        std::vector<float> descriptor = computeDescriptors(file_path, executionTime);
        if (descriptor.empty()) {
            std::cout << "Vector is empty" << std::endl;
        } else {
            int label = 0;
            std::vector<float> descriptor1;
            int length = static_cast<int>(descriptor.size());
            std::cout << "Dimension of descriptor: " << length << std::endl;
            descriptor1.push_back(length);
            saveDescriptorAsCSV(descriptor, "descriptor.csv", label, executionTime);
            descriptor.clear(); // Clear the vector
            descriptor1.clear();
        }

    }


    return 0;
}


Overwriting gradient_computation.cu


In [ ]:
# Experiment parameters
CELL_SIZES=(64)
BLOCK_SIZES=(2)
NUM_BINS=(9)
OUTPUT_FILE_BASENAME="descriptor"
DIM_IMG=224

In [13]:
!nvcc gradient_computation.cu -o gradient_computation `pkg-config --cflags --libs opencv4`

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is

In [14]:
!./gradient_computation 64 2 9 descriptor.csv 224

Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/10.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/108.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/117.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/100.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/1.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/113.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/111.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/11.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Comput

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

# Assuming df is your DataFrame with features and labels
# Splitting data into features (X) and labels (y)

df = pd.read_csv("descriptor.csv")
df1 = pd.read_csv("descriptor_cell64_block2_bins9_20240730_230305.csv")
new_value = 0

df.iloc[-1, df.columns.get_loc('label')] = new_value

X = df.drop(columns=['label'])  # Replace 'target_column' with the name of your target column
y = df['label']


In [18]:
df

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x137,x138,x139,x140,x141,x142,x143,x144,label,Exec Time
0,0.707517,0.095859,0.090213,0.234234,0.163953,0.073105,0.075206,0.284448,0.555370,0.362358,...,0.358775,0.494454,0.258048,0.194932,0.174673,0.186705,0.243515,0.281776,1,0.148338
1,0.822687,0.280493,0.176226,0.297993,0.238534,0.083310,0.075005,0.112680,0.206139,0.643267,...,0.298940,0.304530,0.278457,0.226737,0.212460,0.187485,0.210920,0.272148,1,0.000072
2,0.522358,0.266394,0.422266,0.334273,0.378603,0.326701,0.207820,0.222299,0.153132,0.117661,...,0.202334,0.213733,0.218960,0.385171,0.468673,0.420357,0.208378,0.157847,1,0.000058
3,0.177814,0.058250,0.057333,0.241276,0.942931,0.088277,0.033646,0.035817,0.064515,0.079319,...,0.178250,0.235146,0.307402,0.475547,0.530087,0.408563,0.328167,0.166671,1,0.000055
4,0.655013,0.074662,0.070753,0.116034,0.367965,0.230707,0.191667,0.262129,0.502834,0.213235,...,0.341647,0.330521,0.282466,0.636378,0.250465,0.150787,0.206227,0.280177,1,0.000055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,0.619156,0.112077,0.124535,0.143167,0.315157,0.187211,0.187977,0.354883,0.521948,0.219930,...,0.231810,0.200745,0.218844,0.801915,0.276247,0.213413,0.110165,0.091729,0,0.000058
917,0.034784,0.008650,0.026957,0.127687,0.983218,0.120785,0.018789,0.004897,0.000790,0.016739,...,0.134315,0.165870,0.685969,0.307762,0.240117,0.188144,0.093444,0.319522,0,0.000056
918,0.610379,0.230206,0.147448,0.156651,0.351962,0.496825,0.264001,0.210136,0.208797,0.379372,...,0.141768,0.168427,0.219934,0.461557,0.616182,0.459441,0.263488,0.124403,0,0.000051
919,0.447218,0.311607,0.307339,0.341800,0.374051,0.241301,0.171222,0.398952,0.324027,0.206929,...,0.142378,0.483637,0.484039,0.592594,0.155466,0.177496,0.077784,0.053165,0,0.000059


In [19]:
df1

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x137,x138,x139,x140,x141,x142,x143,x144,label,Exec Time
0,0.307413,0.263576,0.178032,0.272125,0.620107,0.379936,0.360098,0.212224,0.163353,0.614390,...,0.192993,0.416075,0.504421,0.394215,0.338098,0.226097,0.311952,0.204386,1,0.009683
1,0.655013,0.074662,0.070753,0.116034,0.367965,0.230707,0.191667,0.262129,0.502834,0.355626,...,0.199545,0.245505,0.262936,0.724077,0.261280,0.125947,0.119386,0.213449,1,0.009542
2,0.707517,0.095859,0.090213,0.234234,0.163953,0.073105,0.075206,0.284448,0.555370,0.725811,...,0.318048,0.435401,0.180232,0.125543,0.138812,0.171537,0.231036,0.278024,1,0.009195
3,0.177814,0.058250,0.057333,0.241276,0.942931,0.088277,0.033646,0.035817,0.064515,0.139902,...,0.072119,0.077173,0.147682,0.584836,0.671063,0.317192,0.201668,0.102658,1,0.008368
4,0.959483,0.025849,0.016280,0.230602,0.105866,0.034942,0.028507,0.042331,0.101236,0.862538,...,0.225859,0.517545,0.415059,0.261776,0.239796,0.343701,0.383408,0.184780,1,0.009388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,0.489571,0.080354,0.084495,0.221868,0.127130,0.057922,0.474742,0.404621,0.537479,0.977904,...,0.356147,0.134431,0.063069,0.089058,0.073882,0.209415,0.260708,0.181602,0,0.007048
917,0.680131,0.033860,0.021887,0.048971,0.159495,0.067860,0.095489,0.148166,0.687228,0.986685,...,0.006349,0.009614,0.007732,0.027001,0.173730,0.962419,0.126490,0.116647,0,0.005993
918,0.562015,0.020816,0.029778,0.059990,0.813070,0.049105,0.017453,0.026056,0.121421,0.725017,...,0.199491,0.221668,0.350963,0.464609,0.426461,0.199707,0.123030,0.175297,0,0.008121
919,0.333765,0.074165,0.088704,0.131458,0.880665,0.095221,0.049096,0.053708,0.260803,0.448486,...,0.127600,0.184511,0.111548,0.782114,0.404419,0.094450,0.130796,0.216737,0,0.006463


In [ ]:
y

0    1
1    1
2    1
3    0
Name: label, dtype: int64

In [ ]:
# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training a Linear Support Vector Machine
svm = LinearSVC()
svm.fit(X_train, y_train)

# Making predictions
y_pred = svm.predict(X_test)

# Evaluating accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6857142857142857
